In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from scipy.ndimage import correlate
from tqdm import tqdm as tqdm

%matplotlib inline

In [ ]:
savingName = 'fancy_light'

In [ ]:
vidObj = cv.VideoCapture("./videos/%s.mp4" % savingName )
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('./output/%s.avi' % savingName,fourcc, 30.0, (640,480))

In [ ]:
imgArray = []
maskArray = []

In [ ]:
# success, img1 = vidObj.read()
# img = cv.resize(img1, dsize=(640,480), interpolation=cv.INTER_LINEAR)

# mask = np.zeros(img.shape[:2],np.uint8)

# bgdModel = np.zeros((1,65),np.float64)
# fgdModel = np.zeros((1,65),np.float64)
# rect = (320,180,200,300)



# cv.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT);
# front_mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
# fore = img*front_mask[:,:,np.newaxis]

# plt.imshow(cv.cvtColor(fore, cv.COLOR_BGR2RGB)),plt.colorbar(),plt.show();

In [ ]:
%%time

success = 1
i=1

success, img1 = vidObj.read()
while success:
    img = cv.resize(img1, dsize=(640,480), interpolation=cv.INTER_LINEAR)
    mask = np.zeros(img.shape[:2],np.uint8)
    
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (320,180,200,300)
    
    cv.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT);
    front_mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    fore = img*front_mask[:,:,np.newaxis]
    out.write(fore)
    print(i)
    i +=1
    
    imgArray.append(img)
    maskArray.append(mask)
    
    success, img1 = vidObj.read()

vidObj.release()
out.release()
print("Ended")

In [ ]:
maskArray = np.asarray(maskArray)
imgArray = np.asarray(imgArray)

maskArray.shape

In [ ]:
filt = np.ones([10,3,3])

In [ ]:
# 1 in foreground
mask = np.where((maskArray==2)|(maskArray==0),0,1).astype('uint8')

In [ ]:
c = correlate(mask,filt,mode = 'nearest')


In [ ]:

# if number of pixel in neighbourhood >= 10
maskNew = np.where((c >= 60),1,0).astype('uint8')


In [ ]:
newout = cv.VideoWriter('./output/smooth%s.avi' % savingName,fourcc, 30.0, (640,480))

In [ ]:
imgArray.shape

In [ ]:
img_old = imgArray
mask_old = maskNew

maskNew = maskNew[:120,:,:]
imgArray = imgArray[:120,:,:,:]


In [ ]:
for i in range(imgArray.shape[0]):
    img = imgArray[i,:,:,:]
    front_mask = maskNew[i,:,:]
    fore = img*front_mask[:,:,np.newaxis]
    newout.write(fore)
newout.release()

In [ ]:
## Smoothing
blurvideo = cv.VideoWriter('./output/blur%s.avi' % savingName,fourcc, 30.0, (640,480))

In [ ]:
for i in tqdm(range(imgArray.shape[0])):
    img = imgArray[i,:,:,:]
    front_mask = maskNew[i,:,:]
    fore = img * front_mask[:,:,np.newaxis]
    back = img - fore    
    gamma = 3.0
    imgTemp = np.power(img,gamma)
    maxEle = np.amax(imgTemp)
    gamCorrImg=np.divide(imgTemp,maxEle)
    gamCorrImg=gamCorrImg*255
    gamCorrImg=gamCorrImg.astype(np.uint8)
    K = 32
    x = np.arange(0, K)
    y = np.arange(0, K)
    arr = np.zeros((y.size, x.size))
    cx = K/2 - 1.
    cy = K/2 - 1.
    r = K/2 - 1.
    maska = (x[np.newaxis,:]-cx)**2 + (y[:,np.newaxis]-cy)**2 < r**2
    arr[maska] = 1.
    kern=np.divide(arr,np.sum(arr))
    tImg=np.zeros(np.shape(gamCorrImg))
    tImg[:,:,0]=cv.filter2D(gamCorrImg[:,:,0],-1,kern)
    tImg[:,:,1]=cv.filter2D(gamCorrImg[:,:,1],-1,kern)
    tImg[:,:,2]=cv.filter2D(gamCorrImg[:,:,2],-1,kern)
    tImg=tImg.astype(np.uint8)
    bImg=np.power(tImg,(1.0/gamma))
    maxE = np.amax(bImg)
    bImg=np.divide(bImg,(1.0*maxE))
    bImg=bImg*255
    bImg=bImg.astype(np.uint8)
    tImg2=np.zeros(np.shape(img))
    tImg2[:,:,0]=cv.filter2D(img[:,:,0],-1,kern)
    tImg2[:,:,1]=cv.filter2D(img[:,:,1],-1,kern)
    tImg2[:,:,2]=cv.filter2D(img[:,:,2],-1,kern)
    bImg2=tImg2.astype(np.uint8)
    bImg3 = np.maximum(bImg,bImg2)
    imgNew = fore + bImg3 * (1 - front_mask[:,:,np.newaxis])
    blurvideo.write(imgNew)
blurvideo.release()   

In [ ]:
normalVideo = cv.VideoWriter('./output/normal%s.avi' % savingName,fourcc, 30.0, (640,480))

In [ ]:
for i in tqdm(range(imgArray.shape[0])):
    img = imgArray[i,:,:,:]
    normalVideo.write(img)
normalVideo.release()   

In [ ]:
a = 2
a